In [36]:
import findspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import isnan, when, count, col


findspark.init('/Users/AzeezA/opt/spark-3.2.0/')

conf = SparkConf().setMaster("local").setAppName("planes")
sc = SparkContext.getOrCreate(conf=conf)

spark = SparkSession.builder.getOrCreate()

In [2]:
planes = sc.textFile("/Users/AzeezA/Google\ Drive/Life/2021/UPM/Big\ Data/Planes/1988.csv.bz2")
# planes = sc.textFile("hdfs://localhost:9000/temp/1988.csv.bz2")  //if file exists in HDFS use this line


print()
ten_lines = planes.take(10)
for l in ten_lines:
    print(l)


Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
1988,1,9,6,1348,1331,1458,1435,PI,942,NA,70,64,NA,23,17,SYR,BWI,273,NA,NA,0,NA,0,NA,NA,NA,NA,NA
1988,1,10,7,1334,1331,1443,1435,PI,942,NA,69,64,NA,8,3,SYR,BWI,273,NA,NA,0,NA,0,NA,NA,NA,NA,NA
1988,1,11,1,1446,1331,1553,1435,PI,942,NA,67,64,NA,78,75,SYR,BWI,273,NA,NA,0,NA,0,NA,NA,NA,NA,NA
1988,1,12,2,1334,1331,1438,1435,PI,942,NA,64,64,NA,3,3,SYR,BWI,273,NA,NA,0,NA,0,NA,NA,NA,NA,NA
1988,1,13,3,1341,1331,1503,1435,PI,942,NA,82,64,NA,28,10,SYR,BWI,273,NA,NA,0,NA,0,NA,NA,NA,NA,NA
1988,1,14,4,1332,1331,1447,1435,PI,942,NA,75,64,NA,12,1,SYR,BWI,273,NA,NA,0,NA,0,NA,NA,NA,NA,NA
1988,1,15,5,1331,1331,1434,1435,PI,942,NA,63,64,NA,-1,0,SYR,BWI,273,NA,NA,0,NA,0,NA,NA,NA,NA,NA
1988,1,16,6,1327,1331,1427,

In [3]:
print(planes.take(1)[0].split(','))

['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime', 'ArrTime', 'CRSArrTime', 'UniqueCarrier', 'FlightNum', 'TailNum', 'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay', 'DepDelay', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut', 'Cancelled', 'CancellationCode', 'Diverted', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']


In [4]:
column_names = planes.take(1)[0].split(',')

header = planes.first()
planes_no_header = planes.filter(lambda line: line != header)

split_planes = planes_no_header.map(lambda x : x.split(','))

full_planes_df = spark.createDataFrame(split_planes, schema = column_names)
print((full_planes_df.count(), len(full_planes_df.columns)))

(5202096, 29)


In [134]:
%%time
print(full_planes_df.columns)
print(len(full_planes_df.columns))
print()
print()
# print(planes_df.dtypes)

excluded_cols = ['ArrTime', 'ActualElapsedTime', 'AirTime', 'TaxiIn', 'Diverted',
                   'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']
int_cols = ['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime', 'CRSArrTime', 'FlightNum', 
            'TailNum', 'CRSElapsedTime', 'DepDelay', 'Distance', 'TaxiOut', 'Cancelled', 'CancellationCode', 'ArrDelay']
cat_cols = ['UniqueCarrier', 'Orig', 'Dest']


target_col = 'ArrDelay'

target_df = full_planes_df.select(target_col)
planes_df = full_planes_df.drop(*excluded_cols)

for i in int_cols:
    planes_df = planes_df.withColumn(i , planes_df[i].cast(IntegerType()))


full_planes_df.createOrReplaceTempView("FULL_DF")
planes_df.createOrReplaceTempView("FLIGHT_LIST")
target_df.createOrReplaceTempView("TRGT_LIST")

print('\n'.join(str(line) for line in planes_df.dtypes))

print(planes_df.select(planes_df.columns[:6]).summary().show())
print(planes_df.select(planes_df.columns[6:12]).summary().show())
print(planes_df.select(planes_df.columns[12:19]).summary().show())

['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime', 'ArrTime', 'CRSArrTime', 'UniqueCarrier', 'FlightNum', 'TailNum', 'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay', 'DepDelay', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut', 'Cancelled', 'CancellationCode', 'Diverted', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']
29


('Year', 'int')
('Month', 'int')
('DayofMonth', 'int')
('DayOfWeek', 'int')
('DepTime', 'int')
('CRSDepTime', 'int')
('CRSArrTime', 'int')
('UniqueCarrier', 'string')
('FlightNum', 'int')
('TailNum', 'int')
('CRSElapsedTime', 'int')
('ArrDelay', 'int')
('DepDelay', 'int')
('Origin', 'string')
('Dest', 'string')
('Distance', 'int')
('TaxiOut', 'int')
('Cancelled', 'int')
('CancellationCode', 'int')
+-------+-------+-----------------+-----------------+------------------+------------------+------------------+
|summary|   Year|            Month|       DayofMonth|         DayOfWeek|           DepTime|    

### Average Delays
#### From Arrivals

In [136]:
%%time
spark.sql("""SELECT
                Origin, round(avg(ArrDelay), 2) as avg_delay, count(ArrDelay) from FULL_DF group by 1 order by 2 desc;""").show(n= 50)

+------+---------+---------------+
|Origin|avg_delay|count(ArrDelay)|
+------+---------+---------------+
|   ROP|    178.0|              1|
|   ACV|    33.79|             19|
|   RDD|     25.0|             19|
|   YAP|     20.9|            235|
|   BFI|     20.0|              1|
|   HDN|    17.54|            453|
|   ROR|    13.57|            368|
|   DET|    13.12|           2002|
|   JNU|    11.53|           3349|
|   JFK|     11.5|          51485|
|   CLT|    11.32|         101093|
|   SIT|    10.71|           1371|
|   PSG|    10.65|            664|
|   MDW|    10.55|          19447|
|   PIT|    10.29|         118068|
|   ERI|    10.12|           2285|
|   GUC|     9.79|            153|
|   DAY|     9.63|          35562|
|   KTN|     9.62|           2211|
|   VPS|     9.59|           1262|
|   ACY|     9.59|            447|
|   MHT|     9.28|           2800|
|   GST|     9.13|             86|
|   GUM|     9.11|           1733|
|   MSP|     9.02|          95265|
|   SPN|     8.98|  

#### From Departures

In [137]:
%%time
spark.sql("""SELECT
                Dest, round(avg(ArrDelay), 2), count(ArrDelay) from FULL_DF group by 1 order by 2 desc;""").show(n= 50)

+----+-----------------------+---------------+
|Dest|round(avg(ArrDelay), 2)|count(ArrDelay)|
+----+-----------------------+---------------+
| RDD|                  34.61|             18|
| ACV|                  31.67|             18|
| YAP|                  17.68|            228|
| ACY|                  16.11|            433|
| DET|                  13.67|           1959|
| ROR|                  12.53|            355|
| ERI|                  11.99|           2175|
| EVV|                  11.22|           2543|
| SPN|                  10.89|           1116|
| FNT|                  10.76|           1331|
| EWR|                  10.65|         126216|
| AVL|                  10.59|           2292|
| KTN|                  10.49|           2183|
| CDV|                  10.39|            645|
| SIT|                  10.38|           1370|
| EUG|                  10.36|           3567|
| UCA|                  10.27|           1028|
| PSG|                  10.21|            650|
| ALB|       

#### Arrival -> Departure Pairs (aka Average delay for a specific route)

In [135]:
%%time
route_delays = spark.sql("""SELECT
                Origin, Dest, round(avg(ArrDelay), 2), count(ArrDelay) as rt_count from FULL_DF where rt_count > 15 group by 1,2 order by 3 desc;""")

AnalysisException: cannot resolve 'rt_count' given input columns: [full_df.ActualElapsedTime, full_df.AirTime, full_df.ArrDelay, full_df.ArrTime, full_df.CRSArrTime, full_df.CRSDepTime, full_df.CRSElapsedTime, full_df.CancellationCode, full_df.Cancelled, full_df.CarrierDelay, full_df.DayOfWeek, full_df.DayofMonth, full_df.DepDelay, full_df.DepTime, full_df.Dest, full_df.Distance, full_df.Diverted, full_df.FlightNum, full_df.LateAircraftDelay, full_df.Month, full_df.NASDelay, full_df.Origin, full_df.SecurityDelay, full_df.TailNum, full_df.TaxiIn, full_df.TaxiOut, full_df.UniqueCarrier, full_df.WeatherDelay, full_df.Year]; line 2 pos 102;
'Sort [unresolvedordinal(3) DESC NULLS LAST], true
+- 'Aggregate [unresolvedordinal(1), unresolvedordinal(2)], ['Origin, 'Dest, unresolvedalias('round('avg('ArrDelay), 2), None), 'count('ArrDelay) AS rt_count#12182]
   +- 'Filter ('rt_count > 15)
      +- SubqueryAlias full_df
         +- View (`FULL_DF`, [Year#0,Month#1,DayofMonth#2,DayOfWeek#3,DepTime

In [24]:
print(route_delays.show())

+------+----+-----------------------+---------------+
|Origin|Dest|round(avg(ArrDelay), 2)|count(ArrDelay)|
+------+----+-----------------------+---------------+
|   SYR| DEN|                  487.0|              1|
|   ONT| SAT|                  322.0|              1|
|   MDT| FLL|                  260.0|              1|
|   ANC| SIT|                  219.0|              1|
|   ROP| GUM|                  178.0|              1|
|   IND| FWA|                  101.0|              1|
|   CVG| MDT|                   93.0|              1|
|   BOS| PVD|                   80.0|              1|
|   CHS| PBI|                   76.0|              1|
|   ICT| CPR|                   72.0|              1|
|   BHM| MOB|                   67.0|              1|
|   ANC| HNL|                   63.5|              2|
|   EWR| ONT|                   57.0|              7|
|   TUL| MSP|                   56.0|              1|
|   TUS| ONT|                   53.0|              1|
|   YKM| SFO|               

In [45]:
full_planes_df.select([count(when(isnan(c), c)).alias(c) for c in full_planes_df.columns]).show(vertical = True)


-RECORD 0----------------
 Year              | 0   
 Month             | 0   
 DayofMonth        | 0   
 DayOfWeek         | 0   
 DepTime           | 0   
 CRSDepTime        | 0   
 ArrTime           | 0   
 CRSArrTime        | 0   
 UniqueCarrier     | 0   
 FlightNum         | 0   
 TailNum           | 0   
 ActualElapsedTime | 0   
 CRSElapsedTime    | 0   
 AirTime           | 0   
 ArrDelay          | 0   
 DepDelay          | 0   
 Origin            | 0   
 Dest              | 0   
 Distance          | 0   
 TaxiIn            | 0   
 TaxiOut           | 0   
 Cancelled         | 0   
 CancellationCode  | 0   
 Diverted          | 0   
 CarrierDelay      | 0   
 WeatherDelay      | 0   
 NASDelay          | 0   
 SecurityDelay     | 0   
 LateAircraftDelay | 0   



In [23]:
spark.sql("""SELECT 
          count(distinct(UniqueCarrier)) as distinct_carriers from FLIGHT_LIST""").show(vertical = True)

-RECORD 0----------------
 distinct_carriers | 14  



In [25]:
spark.sql("""SELECT 
          count(distinct(Origin)) as distinct_origin, count(distinct(Dest)) as distinct_destination from FLIGHT_LIST""").show(vertical = True)

-RECORD 0-------------------
 distinct_origin      | 238 
 distinct_destination | 236 



In [24]:
spark.sql("""SELECT 
          * from FLIGHT_LIST""").show(vertical = True, n = 1)

-RECORD 0----------------
 Year             | 1988 
 Month            | 1    
 DayofMonth       | 9    
 DayOfWeek        | 6    
 DepTime          | 1348 
 CRSDepTime       | 1331 
 CRSArrTime       | 1435 
 UniqueCarrier    | PI   
 FlightNum        | 942  
 TailNum          | NA   
 CRSElapsedTime   | 64   
 DepDelay         | 17   
 Origin           | SYR  
 Dest             | BWI  
 Distance         | 273  
 TaxiOut          | NA   
 Cancelled        | 0    
 CancellationCode | NA   
only showing top 1 row



### Linear Regression Model

In [46]:
planes_df.dtypes

[('Year', 'int'),
 ('Month', 'int'),
 ('DayofMonth', 'int'),
 ('DayOfWeek', 'int'),
 ('DepTime', 'int'),
 ('CRSDepTime', 'int'),
 ('CRSArrTime', 'int'),
 ('UniqueCarrier', 'string'),
 ('FlightNum', 'int'),
 ('TailNum', 'int'),
 ('CRSElapsedTime', 'int'),
 ('ArrDelay', 'int'),
 ('DepDelay', 'int'),
 ('Origin', 'string'),
 ('Dest', 'string'),
 ('Distance', 'int'),
 ('TaxiOut', 'int'),
 ('Cancelled', 'int'),
 ('CancellationCode', 'int')]

In [124]:
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler


training_cols = ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime', 'CRSArrTime', 'CRSElapsedTime',
                'DepDelay', 'Distance']

target_col = 'ArrDelay'


features = planes_df.select(training_cols + [target_col] )
outputCol = 'features'

df_va = VectorAssembler(inputCols = training_cols, outputCol = outputCol, handleInvalid="skip")
features = df_va.transform(features)


#5202096
print(features.count())
print(features.printSchema())
#5126498
features = features.na.drop(subset = training_cols + [target_col, outputCol])

model_data = features.select([outputCol, target_col])
print("here")
print(features.count())

# # # Use scaler of choice; here Standard scaler is used
# scalar = StandardScaler(inputCol="vectored_features", outputCol="scaled_vectored_features")
# scaled_features = scalar.fit(features)

model_data.limit(10).show()


5140882
root
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: integer (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- CRSElapsedTime: integer (nullable = true)
 |-- DepDelay: integer (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- ArrDelay: integer (nullable = true)
 |-- features: vector (nullable = true)

None
here
5126498
+--------------------+--------+
|            features|ArrDelay|
+--------------------+--------+
|[1.0,9.0,6.0,1348...|      23|
|[1.0,10.0,7.0,133...|       8|
|[1.0,11.0,1.0,144...|      78|
|[1.0,12.0,2.0,133...|       3|
|[1.0,13.0,3.0,134...|      28|
|[1.0,14.0,4.0,133...|      12|
|[1.0,15.0,5.0,133...|      -1|
|[1.0,16.0,6.0,132...|      -8|
|[1.0,17.0,7.0,133...|       5|
|[1.0,18.0,1.0,134...|      44|
+--------------------+--------+



In [127]:
%%time
from pyspark.ml.regression import GeneralizedLinearRegression

(train_data, test_data) = model_data.randomSplit([0.8, 0.2])

glr=GeneralizedLinearRegression(labelCol=target_col,family="gaussian",maxIter=10,regParam=0.3)
model = glr.fit(train_data)

print("Coefficients: ", model.coefficients)
print("Intercept: ", model.intercept)

print(model.summary)

Coefficients:  [-0.0968474021005805,-0.01090305148691135,-0.26903680002202596,0.0041071381987276365,-0.004336729171509588,0.000584549619013383,-0.07724317785051543,0.8957983589395305,0.008226942559914047]
Intercept:  5.012811209506352
Coefficients:
       Feature Estimate Std Error   T Value P Value
   (Intercept)   5.0128    0.0350  143.4121  0.0000
         Month  -0.0968    0.0019  -49.7577  0.0000
    DayofMonth  -0.0109    0.0008  -14.3753  0.0000
     DayOfWeek  -0.2690    0.0034  -80.1339  0.0000
       DepTime   0.0041    0.0001   68.4378  0.0000
    CRSDepTime  -0.0043    0.0001  -72.1445  0.0000
    CRSArrTime   0.0006    0.0000   24.3569  0.0000
CRSElapsedTime  -0.0772    0.0005 -160.8974  0.0000
      DepDelay   0.8958    0.0003 2732.5901  0.0000
      Distance   0.0082    0.0001  139.4971  0.0000

(Dispersion parameter for gaussian family taken to be 185.1975)
   Null deviance: 2225752549.4102 on 4102586 degrees of freedom
Residual deviance: 759788855.4988 on 4102586 degre

#### Site with really good examples https://towardsdatascience.com/building-a-linear-regression-with-pyspark-and-mllib-d065c3ba246a

In [130]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol = 'features', labelCol=target_col, maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_data)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

Coefficients: [-0.03560565238442641,0.0,-0.15139064142555256,0.0,0.0,0.0,-0.007996341233784475,0.896223376264238,0.0]
Intercept: 2.3172655781620057
RMSE: 13.679080
r2: 0.655098


In [133]:
%%time
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator

gbt = GBTRegressor(featuresCol = 'features', labelCol = target_col, maxIter=10)
gbt_model = gbt.fit(train_data)
gbt_predictions = gbt_model.transform(test_data)
# gbt_predictions.select('prediction', target_col, 'features').show(5)

gbt_evaluator = RegressionEvaluator(
    labelCol=target_col, predictionCol="prediction", metricName="rmse")
rmse = gbt_evaluator.evaluate(gbt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 13.8532
CPU times: user 99 ms, sys: 45.3 ms, total: 144 ms
Wall time: 12min 57s
